In [2]:
import tkinter as tk
from tkinter import ttk
import tkinter.font as tkfont
import sys

# COM / Audio Imports
from comtypes import CLSCTX_ALL, CoCreateInstance, cast, CoInitialize
from ctypes import POINTER
from pycaw.pycaw import IAudioEndpointVolume, IMMDeviceEnumerator
from pycaw.constants import CLSID_MMDeviceEnumerator

# --- 1. Audio Setup (Microphone Specific) ---

# Ensure COM is initialized (important for Jupyter environments)
CoInitialize()

# Constants for Audio Endpoint
# eRender = 0 (Speakers), eCapture = 1 (Microphone)
eCapture = 1 
eConsole = 0

try:
    device_enumerator = CoCreateInstance(CLSID_MMDeviceEnumerator, IMMDeviceEnumerator, clsctx=CLSCTX_ALL)
    # GetDefaultAudioEndpoint(data_flow, role) -> 1 = Capture (Mic), 0 = Console
    device = device_enumerator.GetDefaultAudioEndpoint(eCapture, eConsole)
    interface = device.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))
except Exception as e:
    print(f"Error initializing Audio: {e}")
    print("Ensure a microphone is connected.")
    # Fallback to prevent crash if no mic found, though functionality will fail
    volume = None

_ignore_slider_callback = False

# --- 2. Logic Functions ---

def get_current_volume_percent():
    if not volume: return 0
    try:
        # GetMasterVolumeLevelScalar returns 0.0 to 1.0
        return int(round(volume.GetMasterVolumeLevelScalar() * 100.0))
    except:
        return 0

def safe_cfg(widget, **kwargs):
    """Safely configure a widget if it still exists."""
    try:
        if widget and widget.winfo_exists():
            widget.config(**kwargs)
    except Exception:
        pass

def set_volume_percent(percent):
    global _ignore_slider_callback
    if not volume: return

    percent = max(0, min(100, int(round(percent))))
    _ignore_slider_callback = True
    
    try:
        volume.SetMasterVolumeLevelScalar(percent / 100.0, None)
    except Exception as e:
        print(f"Error setting volume: {e}")

    # Update UI immediately
    try:
        if 'volume_slider' in globals() and volume_slider.winfo_exists():
            volume_slider.set(percent)
        if 'volume_progress' in globals() and volume_progress.winfo_exists():
            volume_progress['value'] = percent
        if 'volume_value_label' in globals() and volume_value_label.winfo_exists():
            safe_cfg(volume_value_label, text=f"{percent}%")
        update_mic_icon(percent)
    finally:
        # Re-enable slider callback after a tiny delay
        root.after(10, lambda: set_ignore_false())

def set_ignore_false():
    global _ignore_slider_callback
    _ignore_slider_callback = False

def on_slider_change(val):
    global _ignore_slider_callback
    if _ignore_slider_callback:
        return
    try:
        v = float(val)
        set_volume_percent(v) # Reuse set_volume logic
    except Exception:
        pass

def toggle_mute():
    if not volume: return
    try:
        is_muted = bool(volume.GetMute())
        volume.SetMute(0 if is_muted else 1, None)
        update_mute_button_text()
        
        # Force UI refresh
        cur = get_current_volume_percent()
        update_mic_icon(cur)
    except Exception:
        pass

def update_mute_button_text():
    if not volume: return
    try:
        if 'mute_button' in globals() and mute_button.winfo_exists():
            current_mute = bool(volume.GetMute())
            # Visual feedback on the button
            if current_mute:
                mute_button.config(text="Unmute (Mic Off)", style="Accent.TButton") # Highlight when muted
            else:
                mute_button.config(text="Mute Mic", style="Outline.TButton")
    except Exception:
        pass

def volume_up(step=5):
    cur = get_current_volume_percent()
    set_volume_percent(min(cur + step, 100))

def volume_down(step=5):
    cur = get_current_volume_percent()
    set_volume_percent(max(cur - step, 0))

def on_key_press(event):
    # Keyboard shortcuts when window is focused
    key = event.keysym.lower()
    if key == 'i' or key == 'up':
        volume_up()
    elif key == 'd' or key == 'down':
        volume_down()
    elif key == 'm':
        toggle_mute()
    elif key == 'q' or key == 'escape':
        root.destroy()

def update_mic_icon(percent):
    try:
        if 'mic_icon_label' in globals() and mic_icon_label.winfo_exists():
            if volume and bool(volume.GetMute()):
                icon = "🔇" # Muted
                color = "#ef4444" # Red
            elif percent == 0:
                icon = "🎤" 
                color = "#64748b" # Grey
            else:
                icon = "🎤"
                color = "#0ea5e9" # Blue/Accent
            
            safe_cfg(mic_icon_label, text=icon, fg=color)
    except Exception:
        pass

# --- 3. Build UI (Tkinter) ---

root = tk.Tk()
root.title("Microphone Manager")
root.geometry("420x280")
root.resizable(False, False)

# Theme Colors (Dark UI)
BG = "#0f1724"     # Very dark blue background
CARD = "#1e293b"   # Card background
ACCENT = "#0ea5e9" # Sky blue
TEXT = "#f1f5f9"   # White-ish
SUBTEXT = "#94a3b8" # Grey text

root.configure(bg=BG)

# Global Style
style = ttk.Style(root)
style.theme_use('clam')

# Fonts
title_font = tkfont.Font(family="Segoe UI", size=14, weight="bold")
big_font = tkfont.Font(family="Segoe UI", size=28) # Icon size
label_font = tkfont.Font(family="Segoe UI", size=10)
btn_font = tkfont.Font(family="Segoe UI", size=10, weight="bold")
value_font = tkfont.Font(family="Segoe UI", size=12, weight="bold")

# Main Container (Card)
card = tk.Frame(root, bg=CARD, bd=0, relief=tk.FLAT)
card.place(relx=0.05, rely=0.05, relwidth=0.9, relheight=0.9)

# Header
header_frame = tk.Frame(card, bg=CARD)
header_frame.pack(fill="x", padx=20, pady=(15, 5))

title_label = tk.Label(header_frame, text="Microphone Level", bg=CARD, fg=TEXT, font=title_font)
title_label.pack(side="left", anchor="center")

mic_icon_label = tk.Label(header_frame, text="🎤", bg=CARD, fg=ACCENT, font=big_font)
mic_icon_label.pack(side="right")

# Progress Bar area
progress_frame = tk.Frame(card, bg=CARD)
progress_frame.pack(fill="x", padx=20, pady=(5, 5))

# Customizing Progressbar color
style.configure("Accent.Horizontal.TProgressbar", troughcolor="#334155", background=ACCENT, bordercolor=CARD, lightcolor=ACCENT, darkcolor=ACCENT)
volume_progress = ttk.Progressbar(progress_frame, orient="horizontal", mode="determinate", length=280, style="Accent.Horizontal.TProgressbar")
volume_progress.pack(side="left", fill="x", expand=True, padx=(0,10), pady=4)

volume_value_label = tk.Label(progress_frame, text=f"0%", bg=CARD, fg=TEXT, font=value_font)
volume_value_label.pack(side="right")

# Slider
slider_frame = tk.Frame(card, bg=CARD)
slider_frame.pack(fill="x", padx=16, pady=(0, 10))

style.configure("Horizontal.TScale", troughcolor="#334155", background=ACCENT)
volume_slider = ttk.Scale(slider_frame, from_=0, to=100, orient="horizontal", command=on_slider_change, style="Horizontal.TScale")
volume_slider.set(0) # Will be updated by poll
volume_slider.pack(fill="x", padx=6, pady=6)

# Buttons
btn_frame = tk.Frame(card, bg=CARD)
btn_frame.pack(fill="x", padx=20, pady=(5, 15))

# Button Styles
style.configure("Accent.TButton", font=btn_font, foreground="white", background="#ef4444", padding=(8,6), borderwidth=0) # Red for muted
style.map("Accent.TButton", background=[('active', '#dc2626')])

style.configure("Outline.TButton", font=btn_font, foreground=TEXT, background="#334155", padding=(8,6), borderwidth=0)
style.map("Outline.TButton", background=[('active', '#475569')])

# Mute is the primary action for mics
mute_button = ttk.Button(btn_frame, text="Mute Mic", command=toggle_mute, style="Outline.TButton")
mute_button.pack(side="left", padx=(0,8), ipadx=6, expand=True, fill="x")

vol_down_button = ttk.Button(btn_frame, text="-5%", command=volume_down, style="Outline.TButton")
vol_down_button.pack(side="left", padx=4, ipadx=4)

vol_up_button = ttk.Button(btn_frame, text="+5%", command=volume_up, style="Outline.TButton")
vol_up_button.pack(side="left", padx=(4,0), ipadx=4)

# Footer help text
footer = tk.Label(card, text='Focus window & press: "i/up" (+), "d/down" (-), "m" (mute)', bg=CARD, fg=SUBTEXT, font=tkfont.Font(family="Segoe UI", size=8))
footer.pack(side="bottom", pady=(0,10))

# --- 4. Polling & Execution ---

# Bind keyboard events
root.bind_all("<Key>", on_key_press)

def poll_volume():
    """Periodically checks system volume in case it's changed externally."""
    try:
        cur = get_current_volume_percent()
        
        # Only update UI if we aren't currently dragging the slider
        global _ignore_slider_callback
        if not _ignore_slider_callback:
            _ignore_slider_callback = True # Prevent feedback loop
            try:
                if volume_slider.winfo_exists():
                    volume_slider.set(cur)
                if volume_progress.winfo_exists():
                    volume_progress['value'] = cur
                if volume_value_label.winfo_exists():
                    safe_cfg(volume_value_label, text=f"{cur}%")
            except:
                pass
            # Re-enable callbacks shortly
            root.after(10, set_ignore_false)
            
        update_mute_button_text()
        update_mic_icon(cur)
        
    except Exception:
        pass
    finally:
        if root.winfo_exists():
            root.after(500, poll_volume)

# Initial state load
update_mute_button_text()
initial_vol = get_current_volume_percent()
update_mic_icon(initial_vol)
volume_slider.set(initial_vol)

# Start polling
root.after(500, poll_volume)

# Run Mainloop
try:
    root.mainloop()
except KeyboardInterrupt:
    root.destroy()